# Proyecto 03 - Sistemas de Recomendación

## Dataset: STEAM

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Son dos archivos, uno de calificaciones y otro de información sobre los juegos.**

En este notebook te dejamos unas celdas para que puedas comenzar a trabajar con este dataset. Sin embargo, **deberás** modificarlas para hacer un mejor manejo de datos. Algunas cosas a las que deberás prestar atención (tal vez no a todas):
1. Tipos de datos: elige tipos de datos apropiados para cada columna.
2. Descartar columnas poco informativas.
3. Guardar en memoria datasets preprocesados para no tener que repetir código que tarde en correr.

### Exploración de datos

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿cuáles son los juegos más populares?¿Y los menos populares?

### Filtro Colaborativo

Deberás implementar un sistema de recomendación colaborativo para este dataset. Ten en cuenta:

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.

Puedes implementar un filtro colaborativo a partir de la similitud coseno o índice de Jaccard. ¿Puedes utilizar los métodos de la librería Surprise? Si no es así, busca implementaciones (por ejemplo, nuevas librerías) que sean apropiadas.

Para comenzar a trabajar, puedes asumir que cada entrada es un enlace entre una persona usuaria y un item, **independientemente** de si la crítica es buena o mala. 

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Cómo harías para ponerle un valor a la calificación?
1. ¿Cómo harías para agregar contenido? Por ejemplo, cuentas con el género, precio, fecha de lanzamiento y más información de los juegos.
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

In [ ]:
import gzip
import pandas as pd
import numpy as np
import itertools
import plotly.express as px
import plotly.graph_objects as go
import missingno as msno
from time import strptime
import datetime
import re
import nltk
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image
from scipy.ndimage import gaussian_gradient_magnitude
from IPython.display import display, SVG

In [ ]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

#### Reviews

Obteniendo todos los datos:

In [ ]:
#contador = 0
data_reviews = []
# Vamos a guardar una de cada 10 reviews para no llenar la memoria RAM. Si pones n = 3, 
# abrira uno de cada tres, y asi.
#n = 2
for l in parse('steam_reviews.json.gz'):
    #if contador%n == 0:
    data_reviews.append(l)
    #else:
    #    pass
    #contador += 1

Recibimos las columnas necesarias para el preprocesamiento y análisis, además agiliza la conversión a DataFrame.

In [ ]:
data_reviews = pd.DataFrame(data_reviews, columns=[
                            'username', 'hours', 'product_id', 'date', 'text'])

Conversión tipos de dato para columna "date":

In [ ]:
data_reviews['date'] = pd.to_datetime(data_reviews['date'])

In [ ]:
data_reviews.dtypes

In [ ]:
print(data_reviews.shape)
data_reviews.head()

In [ ]:
data_reviews.isna().sum()*100/len(data_reviews)

#### Games

In [ ]:
data_games = []
for l in parse('steam_games.json.gz'):
    data_games.append(l)
data_games = pd.DataFrame(data_games)

In [ ]:
data_games.head(2)

## Analisis Exploratorio 

El plan a llevar a cabo consiste en separar los videojuegos de las utilidades, complementos o elementos adicionales para que el sistema de recomendación basado en **filtro colaborativo** sea lo mas útil posible para el usuario, las utilidades son paquetes complementarios a un videojuego que ofrece Steam para brindar una mayor experiencia (Packs de personalización de personajes, sonidos de fondo, ambientación, etc) pero consideramos que no deben aparecer en la lista de recomendación porque quizás sea preferible para el usuario ver los videojuegos listos para instalar o jugar, en vez de ciertos items que requieren de un videojuego ya instalado.  
Se decidio entonces elaborar otro mecanismo de recomendación para estas utilidades, no tan riguroso como el propuesto al inicio, pero que esté provisto con lo necesario, el algoritmo **basado en contenido** es una buena opción.

Para poder realizar esta separación de registros se analizarán las categorías de los videojuegos manualmente al indagar el sitio web; una vez obtenido esto, se filtrará y exportarán los resultados en dos datasets diferentes. Lo mismo sucederá con los reviews, se incluirán en un dataset diferente aquellos reviews sobre las utilidades, además con ésto se logra reducir más la cantidad de reviews actual.

### Preprocesamiento Games

In [ ]:
data_games.head(3)

#### Valores faltantes

In [ ]:
data_games.isna().sum()*100/len(data_games)

Visualización de valores faltantes:

In [ ]:
msno.matrix(data_games)

In [ ]:
data_games.shape

Las columnas a continuación no otorgan valor para el análisis exploratorio, se incluyen algunas con valores faltantes superior al 90% del total de los datos. El resto de columnas no incluidas aqui servirán para un posterior análisis exploratorio.

In [ ]:
data_games.drop(columns=['title', 'discount_price', 'reviews_url',
                         'early_access', 'developer', 'sentiment', 'metascore'], inplace=True)

In [ ]:
data_games.isna().sum()*100/len(data_games)

##### Videjuegos con nombre faltante:

El nombre de la aplicación es importante a la hora de conocer que aplicación se va a recomendar

In [ ]:
data_games[data_games.app_name.isna()]

* Al parecer hay un videojuego que no existe (indice 74), se puede verificar examinando su url la cual no está compuesta por id ni nombre del videojuego.  
* Con el segundo registro obtenido, al acceder a su url (http://store.steampowered.com/app/317160/_/) parece ser una versión Demo gratuita, no aparece el titulo, ni imagenes adjuntas pero tiene reviews de algunos usuarios, en la imagen portada de se logra apreciar su nombre ("Duet").  
Al consultar otro videojuego con este mismo nombre encontramos la versión completa de pago con los datos completos.

In [ ]:
data_games.query('app_name=="Duet"')

Asignando nombre de videojuego faltante:

In [ ]:
data_games.iloc[2580].app_name = 'Duet'

Aplicando dropna() al dataset con valores faltantes por la columna **"app_name"**

In [ ]:
data_games.dropna(subset=['app_name'], inplace=True)
data_games.isna().sum()

##### Videojuegos con ID faltante

El ID del videojuego también es un identificador importante y no se debe ignorar.

In [ ]:
data_games[data_games.id.isna()]

Vemos que la url contiene el id faltante y en este caso se obtuvo un solo registro, con un simple _"copy - paste"_ de este id y conociendo el índice se podría solucionar, pero en caso de haber obtenido multiples registros la tarea sería mas tediosa si se hace manualmente. Crear una iteración sobre éstos videojuegos para completar el id es lo mas viable en este caso:

Split basico a una url para obtener el id:

In [ ]:
'http://store.steampowered.com/app/200260'.split('app/')[1].split('/')[0]

Aplicando la estrategia en un loop:

In [ ]:
games_id_nan = data_games[data_games.id.isna()].index
for idx in games_id_nan:
    url = data_games.loc[idx].url
    id_game = url.split('app/')[1].split('/')[0]
    data_games.loc[idx].id = id_game

In [ ]:
data_games[data_games.index==30961]

In [ ]:
data_games.isna().sum()

In [ ]:
data_games.shape

#### Videojuegos duplicados

Estrategia para eliminar videojuegos duplicados empezando por aquellos cuyo ID sea el mismo.

In [ ]:
data_games[data_games.duplicated(
    subset=['id'], keep=False)].sort_values(by='publisher')

In [ ]:
data_games.drop_duplicates(subset=['id'], ignore_index=True, inplace=True)

In [ ]:
data_games.shape

In [ ]:
data_games.isna().sum()

#### Filtrado de categorías

Se filtrarán aquellas categorías relacionadas a Hardware, Software, vídeos, etc que se encuentran en este dataset procurando dejar únicamente videojuegos válidos para el sistema de recomendación basado en filtro colaborativo. Los registros exentos permanecerán para una recomendación más simple; creemos que aunque al usuario solo le interesen videojuegos tal vez sienta curiosidad por explorar items adicionales que la plataforma Steam ofrece.

* Filtrando géneros y tags faltantes

Seleccionando videojuegos con género y especificación faltante, con éste filtro removemos una gran cantidad de utilidades y los almacenanos en otro dataframe:

In [ ]:
isNan_genre_tag = data_games.genres.isna() & data_games.specs.isna()
df_no_games = data_games[isNan_genre_tag]
print(df_no_games.shape)

In [ ]:
df_no_games.sample(frac=.2)

In [ ]:
data_games = data_games[~isNan_genre_tag]

In [ ]:
data_games.shape

In [ ]:
data_games.isna().sum()

* Rellenando genres, tags y specs faltantes con una lista vacia, para que las operaciones aplicadas a éstas columnas sea más fácil

In [ ]:
for col in ['genres', 'tags', 'specs']:
    data_games[col].fillna({i: [] for i in data_games.index}, inplace=True)

In [ ]:
data_games.isna().sum()

* Obteniendo categorias únicas para genres, tags y specs en cada registro:

In [ ]:
#Todas las categorías
data_games['categories'] = data_games['genres'] + data_games['tags'] + data_games['specs']

#Obtencion de categorías únicas
data_games['categories'] = [np.unique(cat) for cat in data_games.categories]

#Removiendo caracteres, ej: Single-player singleplayer, Multiplayer, MultiPlayer, Multi-Player
data_games['categories'] = [[re.sub(r'[^\w\s]', '', subcat).lower() for subcat in cat] for cat in data_games.categories]

* Obteniendo categorías válidas (manualmente):

In [ ]:
np.unique(list(itertools.chain(*data_games.categories)))

* Verificando manualmente algunas de éstas categorías: En éste paso, realizamos una tediosa y necesaria labor de identificar cada una de estas categorías y aproximar su validez para un videojuego. Las categorías recoletadas se listan debajo.

In [ ]:
INVALID_CATEGORIES = ['photo editing', 'game development', 'documentary', 'video production', 'tutorial', 'software',
                      'movie', 'software training', 'soundtrack', 'utilities', 'accounting', 'downloadable content']

La categoría "downloadable content", por ejemplo, está relacionada con packs de videojuegos (Verificado al acceder a la URL). Es decir, en este dataset encontramos videojuegos con sus packs respectivos y éstos últimos son elementos que no deberían estar en el sistema de recomendación objetivo.
Para evidenciar lo dicho anteriormente, filtramos por la categoria de ejemplo y notamos a simple vista que algunos registros son complementos de un videojuego con nombre "SNOW" e identificador "244930".

In [ ]:
def getTuplesPandas(search_list):
    return [game for game in data_games.itertuples()
            if np.isin(search_list, game.categories).sum() > 0]

In [ ]:
pd.DataFrame(getTuplesPandas(['downloadable content'])).head(7)

Buscando el videojuego SNOW:

In [ ]:
data_games.query('id=="244930"')

Esto confirma la existencia de muchos complementos, la estrategia mencionada al inicio es separar todos los complementos, utilidades y herramientas adicionales en otro dataset para realizar un análisis con ellos.

* Sumando al dataframe aquellos ítems de esas categorías:

In [ ]:
df_no_games = df_no_games.append(getTuplesPandas(INVALID_CATEGORIES))

In [ ]:
df_no_games.shape

In [ ]:
df_no_games.columns

In [ ]:
df_no_games.drop('Index', axis=1, inplace=True)

In [ ]:
print(df_no_games.shape)
df_no_games.head()

* Filtrando videojuegos:

In [ ]:
data_games = data_games[~data_games.id.isin(df_no_games.id)]

In [ ]:
data_games.shape

Eliminando columnas "specs" y "tags":

In [ ]:
data_games.drop(['specs', 'tags'], inplace=True, axis=1)

Ahora, rellenando con valores faltantes las listas vacías en columna "genres":

In [ ]:
data_games['genres'] = data_games['genres'].apply(lambda x: np.nan if len(x) == 0 else x)

In [ ]:
data_games.isna().sum()

#### Procesamiento de precios

La columna "price" esta conforma por valores numéricos y categóricos:

In [ ]:
category_prices = lambda x: np.unique(np.array([i for i in list(x) if type(i) == str]))

* Categoría de precios

In [ ]:
category_prices(data_games.price)

* Categoría de precios dataset de utilidades:

In [ ]:
category_prices(df_no_games.price)

Creando nuevas categorías para la columna precio, actualmente contamos con valores numéricos y categóricos, la única categoría que se transformará a "paid" es "Starting at $499.00" del dataset de utilidades, el resto quedará como "free".  
Con la siguiente función se procesarán éstas categorías y al mismo tiempo, los valores faltantes se convertirán a "paid" (tal vez algunos sean videojuegos gratuitos pero ésto no tiene gran impacto en el análisis).

In [ ]:
def transform_prices(df):
    df['category_price'] = df.price.apply(
        lambda p: 'paid' if (type(p) == float or p.startswith('Starting')) else 'free')
    df.drop('price', axis=1, inplace=True)

In [ ]:
transform_prices(data_games)
data_games.head()

In [ ]:
transform_prices(df_no_games)
df_no_games.head()

#### Procesamiento de fecha de lanzamiento

En éste apartado, se transformarán algunas fechas del dataset de videojuegos al formato correcto, la columna release_date además de contener fechas de tipo datetime y valores faltantes, tiene valores categóricos del cual es posible convertirlos a datetime gracias a que algunos de ellos siguen el mismo patrón o comportamiento. 
Para el dataset de utilidades, no será necesario éste tipo de tratamiento.

In [ ]:
data_games.release_date.value_counts()

La siguiente función servirá para obtener aquellos valores que no corresponden a fechas válidas para poder examinar los valores categóricos:

In [ ]:
def isDateValid(x): 
    try:
        return datetime.datetime.strptime(x, '%Y-%m-%d')        
    except:
        return False

In [ ]:
invalid_date_games = data_games.query('release_date.notna()', engine='python')
invalid_date_games = pd.DataFrame([i for i in invalid_date_games.itertuples() 
                                   if not isDateValid(i.release_date)])
invalid_date_games.release_date

De las categorías mostradas, las que tienen el nombre del mes (completo o abreviado) y año es posible convertirlas. El nombre de la temporada y el año presentan el mismo caso. **Ejemplos: Nov 2020, November 2020, Winter 2020, Summer 2020...**

* Para empezar, la función a continuación retorna el número del mes basado en el nombre, gracias a una de las librerías incluidas en Python.

In [ ]:
def get_month(month):
    if len(month) > 3:
        return strptime(month, '%B').tm_mon
    else:
        return strptime(month, '%b').tm_mon

In [ ]:
print('Noviembre:', get_month('Nov'))
print('Noviembre:', get_month('November'))
print('Abril:', get_month('Apr'))
print('Abril:', get_month('April'))

Las temporadas se definirarán según el país, en éste caso es Estados Unidos. El diccionario contiene el nombre de la temporada y el mes en que inicia:

In [ ]:
SEASONS = {'Winter': 12, 'Summer': 7, 'Fall': 10, 'Spring': 4}

* Ésta función obtiene una fecha como variable categórica y la transforma a datetime, si no corresponde al patron detectado la convierte a valor faltante. El resto de fechas no están definidas por un comportamiento, por lo tanto no es posible transformarlas una por una, y son mas de 100 categorías.

In [ ]:
def get_date(date_sring):
    try:
        sep = date_sring.split()
        # Primero verifica si es una temporada
        if sep[0] in SEASONS.keys():
            return datetime.datetime(year=int(sep[1]), month=SEASONS.get(sep[0]), day=1).date()

        # Sino, utiliza la función anterior para obtener el número del mes a partir del nombre.
        return datetime.datetime(year=int(sep[1]), month=get_month(sep[0]), day=1).date()
    except:
        # Retorna valor faltante si no se cumplen los dos casos, no es posible hacer más magia!!
        return np.nan

Ejemplo:

In [ ]:
example_date = 'Nov 2020'
print('Noviembre', get_date(example_date))

example_date = 'November 2020'
print('Noviembre', get_date(example_date))

example_date = 'Summer 2020'
print('Verano', get_date(example_date))

example_date = 'Winter 2020'
print('Winter', get_date(example_date))

* Transformando fechas categóricas:

In [ ]:
invalid_date_games.release_date = invalid_date_games.release_date.apply(lambda x: f'{get_date(x)}')
invalid_date_games.release_date

* Aplicando cambios sobre el dataframe original

In [ ]:
invalid_idx = invalid_date_games.Index.values
data_games.loc[invalid_idx,
               'release_date'] = invalid_date_games.release_date.values

* Verificando los cambios en los índices donde previamente existían éstas categorías:

In [ ]:
data_games.loc[invalid_idx].head()

Se ha completado el procedimiento de transformación de fecha de lanzamiento, con ésto es posible realizar un análisis exploratorio que conlleve Series de tiempo.

In [ ]:
del invalid_date_games

In [ ]:
data_games.shape

Finalmente, los valores faltantes resultantes:

In [ ]:
data_games.isna().sum()

In [ ]:
msno.matrix(data_games)

### Preprocesamiento Reviews

In [ ]:
data_reviews.head()

In [ ]:
data_reviews.shape

#### Reviews duplicados (Todos los campos)

Se analiza si existen registros duplicados en el dataset:

In [ ]:
duplicated = data_reviews[data_reviews.duplicated()]
duplicated.shape

Se confirma la existencia registros duplicados, ahora se detalla un usuario aleatorio de éstos registros:

In [ ]:
duplicated.head()

Para probar, se selecciona el usuario **"feel it break"**

In [ ]:
data_reviews[data_reviews.username == 'feel it break']

En los 2 primeros registros de esta búsqueda se evidencia error de los datos, misma cantidad de horas, id de videojuego y fecha. Por lo tanto es razonable eliminar éstos registros.

In [ ]:
data_reviews.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
data_reviews.shape

In [ ]:
data_reviews.isna().sum()

#### Reviews duplicados (Username y product_id)

Habiendo eliminado registros duplicados, nos queda realizar un tratamiento especial a aquellos usuarios que han comentado un videojuego más de una vez, la acción a ejecutar consiste en dejar el ultimo review de un usuario a un videojuego mediante la fecha.

Usuarios con más de un review a un videojuego:

In [ ]:
repeated_reviews = data_reviews[data_reviews.duplicated(
    subset=['username', 'product_id'], keep=False)].sort_values(by='username')

In [ ]:
print('Reviews del mismo usuario al mismo videojuego:', repeated_reviews.shape[0])

In [ ]:
repeated_reviews.head()

El usuario "!?" ha hecho dos comentarios al mismo videojuego (206210), el resultado entonces sería el último comentario realizado.

#### Últimos reviews por fecha

In [ ]:
# Para traer todas las columnas. Tarda mas!!
unrepeated_reviews = data_reviews.sort_values(
    by='date', ascending=False).drop_duplicates(subset=['username', 'product_id'])

# unrepeated_reviews = data_reviews.groupby(
#    ['username', 'product_id'], as_index=False).date.max()

Verificando:

In [ ]:
unrepeated_reviews.query('username=="!?" & product_id=="206210"')

In [ ]:
data_reviews = unrepeated_reviews.copy()

#Liberando espacio en memoria
del repeated_reviews
del unrepeated_reviews

In [ ]:
data_reviews.isna().sum()

In [ ]:
data_reviews.shape

#### Tratamiento Horas faltantes

Vemos que las horas faltantes son muy pocas, con solo calcular el promedio de las horas por cada usuario sería suficiente.

In [ ]:
data_reviews.hours.fillna(data_reviews.groupby('username').hours.transform('mean'), inplace=True)

In [ ]:
data_reviews.isna().sum()

Aún existen horas faltantes, verémos que ocurre:

In [ ]:
data_reviews[data_reviews.hours.isna()]

Explorando algunos usuarios de ésta búsqueda:

In [ ]:
USERNAMES = ['catsquad3', 'cardenas1149', 'MarbleMania']
data_reviews.query('username in @USERNAMES')

Aquí se demuestra que éstos usuarios solo han hecho review a un solo videojuego y no registran horas de juego. Por lo tanto, la única alternativa faltante es rellenarlos con el promedio de horas del id del videojuego en el que estuvieron interesados:

In [ ]:
data_reviews.hours.fillna(data_reviews.groupby('product_id').hours.transform('mean'), inplace=True)

In [ ]:
data_reviews.isna().sum()

In [ ]:
data_reviews[data_reviews.hours.isna()]

Videojuegos que solo tienen un review con horas faltantes. Rellenando con el promedio total de horas:

In [ ]:
data_reviews.hours.fillna(data_reviews.hours.mean(), inplace=True)

In [ ]:
data_reviews.isna().sum()

In [ ]:
data_reviews.shape

#### Separando reviews

Se separarán en datasets diferentes los reviews de videojuegos y utilidades como se ha mencionado en apartados anteriores:

In [ ]:
# Filtrando reviews de videojuegos
data_reviews_game = data_reviews.query('product_id not in @df_no_games.id')

# Filtrando reviews de utilidades (Para exportar)
data_reviews_utilites = data_reviews.query('product_id not in @data_games.id')

In [ ]:
# Dataset listo para exportar
data_reviews = data_reviews_game.copy()

del data_reviews_game

In [ ]:
print('Reviews games:', data_reviews.shape,
      '- Reviews Utilites:', data_reviews_utilites.shape)

### Exportando Datasets

* Dataset de videojuegos:

In [ ]:
directory = 'data_processed_games'
data_reviews.to_csv(f'{directory}/data_reviews_games.csv', index=False)
data_games.to_csv(f'{directory}/data_games.csv', index=False)

* Dataset de utilidades:

In [ ]:
directory = 'data_processed_utilities'
data_reviews_utilites.to_csv(f'{directory}/data_reviews_utilites.csv', index=False)
df_no_games.to_csv(f'{directory}/data_utilities.csv', index=False)

### Lectura

#### Videojuegos procesados:

* Reviews: Se obliga a que la columna username sea tipo string porque existen usuarios cuyo nombre es "null" o "N/A", y Pandas los convierte a NaN. Ejemplo de usuario "null": https://steamcommunity.com/profiles/76561198170981531/

In [ ]:
directory = 'data_processed_games'
data_reviews = pd.read_csv(
    f'{directory}/data_reviews_games.csv', converters={'username': str, 'text': str})
data_reviews.date = pd.to_datetime(data_reviews.date)

* Videojuegos: Al exportar a CSV se pierde el formato de algunos datos, los géneros, tags y specs estaban compuestos por listas, en la sección de Preprocesamiento. En la lectura del archivo, aparecen como String, la siguiente función remueve las comillas simples y los corchetes en los extremos:

In [ ]:
def toList(x):
    return x[1:-1].replace("'", '').split(', ') if (x is not np.nan) else nan

Aquí se convierte cada elemento de tag en una lista:

In [ ]:
data_games = pd.read_csv(f'{directory}/data_games.csv',
                         converters={
                             'genres': lambda x: toList(x),
                             'tags': lambda x: toList(x),
                             'specs': lambda x: toList(x),
                             'categories': lambda x: toList(x)
                         })
data_games.release_date = pd.to_datetime(data_games.release_date)

In [ ]:
data_reviews.dtypes

In [ ]:
data_games.dtypes

In [ ]:
data_games.head()

In [ ]:
data_reviews.head()

In [ ]:
data_reviews.isna().sum()

### Consultas

#### Usuarios únicos

In [ ]:
print(f'{len(data_reviews.username.value_counts())} usuarios únicos')

#### Top 10 videojuegos más populares

In [ ]:
amount_by_productId = data_reviews.groupby('product_id').product_id.count().reset_index(name='amount')

Merge entre videojuegos y reviews:

In [ ]:
reviews_games_amount = pd.merge(amount_by_productId, data_games, left_on='product_id',
                               right_on='id').sort_values(by='amount', ascending=False)
top10 = reviews_games_amount.head(10)

In [ ]:
# Valores para mostrar en la leyenda de cada barra
HOVER_DATA_COLUMNS = ['publisher', 'release_date', 'genres', 'category_price']

In [ ]:
fig = px.bar(top10.sort_values(by='amount'), y='app_name', x='amount', color='amount', title='Videojuegos más populares',
             color_continuous_scale='viridis',
             labels={'app_name': 'Videojuegos',
                     "amount": 'Cantidad de usuarios'},
             hover_data=HOVER_DATA_COLUMNS,
             width=900,
             height=450
             )
fig.update_layout(paper_bgcolor='#FAFAFA',
                  margin=dict(l=20, r=20, t=40, b=50)
                  )
# Jupyter
#fig.show()

# Static
display(SVG(fig.to_image(format='svg')))

In [ ]:
del top10

#### Videojuegos con más tiempo de juego

In [ ]:
hours_by_productId = data_reviews.groupby('product_id', as_index=False).hours.sum()

reviews_games_hours = pd.merge(hours_by_productId, data_games, left_on='product_id',
                               right_on='id').sort_values(by='hours', ascending=False)

hours10 = reviews_games_hours.head(10).sort_values(by='hours')

In [ ]:
fig = px.bar(data_frame=hours10, y='app_name', x='hours', color='hours', title='Videojuegos con mas tiempo de juego',
             color_continuous_scale='viridis',
             labels={'app_name': 'Videojuegos', "hours": 'Horas de juego'},
             hover_data=HOVER_DATA_COLUMNS,
             width=900,
             height=450
             )

fig.update_layout(paper_bgcolor='#FAFAFA',
                  margin=dict(l=20, r=20, t=40, b=10)
                  )
#fig.show()
display(SVG(fig.to_image(format='svg')))

In [ ]:
#Liberando memoria
del hours_by_productId, reviews_games_hours, hours10

#### Género más popular

In [ ]:
all_genres = list(itertools.chain(*data_games.genres))
value, count = np.unique(all_genres, return_counts=True)
order = np.argsort(count)

In [ ]:
fig = px.bar(x=count[order], y=value[order], color=count[order],
             color_continuous_scale='emrld',
             title='Géneros',
             labels={
                 'x': 'Cantidad de videojuegos',
                 'y': 'Generos', 'color': 'Cantidad'},
             width=850,
             height=550,
             # log_x=True,
             )
fig.update_layout(paper_bgcolor='#FAFAFA',
                  margin=dict(l=10, r=20, t=5)
                  )
#fig.show()
display(SVG(fig.to_image(format='svg')))

#### Videojuegos de pago mas jugados

In [ ]:
df_items = data_games.query('category_price=="paid"')
merge_paid_games = pd.merge(amount_by_productId, df_items, left_on='product_id',
                            right_on='id').sort_values(by='amount', ascending=False).head(10)

In [ ]:
fig = px.bar(data_frame=merge_paid_games, x='app_name', y='amount', title='Videojuegos de pago mas jugados',
             labels={'app_name': 'Videojuegos',
                     'amount': 'Cantidad de usuarios'},
             hover_data=HOVER_DATA_COLUMNS,
             color=merge_paid_games.amount, color_continuous_scale='ylorrd',
             width=800,
             height=500
             )

fig.update_layout(paper_bgcolor='#FAFAFA',
                  margin=dict(l=30, r=20, t=40, b=30)
                  )
#fig.show()
display(SVG(fig.to_image(format='svg')))

In [ ]:
del merge_paid_games

#### Videojuegos gratis mas jugados

In [ ]:
df_items = data_games.query('category_price=="free"')
merge_free_games = pd.merge(amount_by_productId, df_items, left_on='product_id',
                            right_on='id').sort_values(by='amount', ascending=False).head(10)

In [ ]:
fig = px.bar(data_frame=merge_free_games, x='app_name', y='amount', title='Videojuegos gratis mas jugados',
             labels={'app_name': 'Videojuegos',
                     'amount': 'Cantidad de usuarios', 'color': 'Cantidad'},
             hover_data=HOVER_DATA_COLUMNS,
             color=merge_free_games.amount, color_continuous_scale='ylorrd',
             width=800,
             height=500)

fig.update_layout(paper_bgcolor='#FAFAFA',
                  margin=dict(l=20, r=20, t=40, b=30)
                  )
#fig.show()
display(SVG(fig.to_image(format='svg')))

In [ ]:
del merge_free_games

#### Porcentaje de usuarios por precio de videojuego

In [ ]:
fig = px.pie(reviews_games_amount,
             names='category_price',
             values='amount',
             title='% Usuarios por precio',
             width=450,
             height=350,
             hole=0.4,
             color_discrete_sequence=('#264653', '#E63946'),
             labels={'category_price': 'Tipo Precio', 'amount': 'Cantidad'}
             )

fig.update_layout(
    margin=dict(t=30, b=0),
    legend=dict(font=dict(size=17)),
    paper_bgcolor='#edf2f4'
)
fig.update_traces(textposition='inside', textinfo='percent+label')
#fig.show()
display(SVG(fig.to_image(format='svg')))

El 81% de los usuarios prefieren pagar por videojuegos.

#### Series de tiempo

##### -> Lanzamiento de videojuegos

In [ ]:
# Creando nueva columna para nombre del mes
games = data_games.assign(
    month_name=data_games.release_date.dt.month_name(),
    year=data_games.release_date.dt.year
)

In [ ]:
# Frecuencia de cada mes
games_month = games.month_name.value_counts()

# Pasando objeto value_counts a DataFrame
games_month = pd.DataFrame(
    {'month_name': games_month.index, 'amount': games_month.values})

# Creando el numero del mes
games_month['n_month'] = pd.to_datetime(
    games_month.month_name, format='%B', errors='coerce').dt.month

# Ordenando por numero de mes
games_month.sort_values(by='n_month', inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=games_month.month_name,
    y=games_month.amount,
    marker={'size': 13, 'opacity': .7}
))
fig.update_layout( 
    paper_bgcolor='#FAFAFA',
    title='Videojuegos / Fechas de lanzamiento',
    autosize=False,
    height=500,
    width=900,
    hoverdistance=100,
    spikedistance=1000,
    xaxis=dict(
        title_text='Meses',
        titlefont=dict(size=18),
        showspikes=True,
        spikethickness=2,
        spikedash="dot",
        spikecolor="#999999",
        spikemode="across"),
    yaxis=dict(title_text="Videojuegos publicados")
)
#fig.show()
display(SVG(fig.to_image(format='svg')))

La mayoría de los videojuegos pueden estar siguiendo una estrategia de marketing lo cual explica que el número de publicaciones se acrecenta enormemente en los últimos meses del año.

##### -> Videojuegos publicados por año

In [ ]:
games_year = games.year.value_counts()
fig = px.bar(x=games_year.index, y=games_year.values, labels={
             'x': 'Año', 'y': 'Cantidad', 'color': 'Cantidad'},
             color=games_year.values,
             title='Videojuegos publicados por año',
             color_continuous_scale='spectral_r',
             width=900,
             height=520,
             log_y=True
             )

fig.update_layout(paper_bgcolor='#FAFAFA',
                  margin=dict(t=50, b=50)
                  )
fig.update_xaxes(rangeslider_visible=True)

#fig.show()
display(SVG(fig.to_image(format='svg')))

La cantidad de videojuegos en escala logarítmica. Se evidencia un aumento en la producción de videojuegos al pasar de los años, sin embargo, los datos muestran un descenso brutal en los últimos dos años. 

In [ ]:
del games, games_month, games_year

##### -> Concentración de usuarios por mes

In [ ]:
users_by_month = data_reviews.assign(month_name=data_reviews.date.dt.month_name())

users_by_month = users_by_month.month_name.value_counts()

users_by_month = pd.DataFrame(
    {'month_name': users_by_month.index, 'amount': users_by_month.values})

users_by_month['month_number'] = pd.to_datetime(users_by_month.month_name,
                                                format='%B', errors='coerce').dt.month

users_by_month.sort_values(by='month_number', inplace=True)
users_by_month

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=users_by_month.month_name,
    y=users_by_month.amount,
    line={'color': '#000000'},
    marker={'size': 13, 'opacity': .7},
))
fig.update_layout(
    paper_bgcolor='#FAFAFA',
    title='Cantidad de usuarios por mes',
    autosize=False,
    width=900,
    height=450,
    hoverdistance=100,
    spikedistance=1000,
    margin=dict(t=50),
    xaxis=dict(
        title_text='Meses',
        titlefont=dict(size=18),
        showspikes=True,
        spikethickness=3,
        spikedash="dot",
        spikecolor="red",
        spikemode="across"),
    yaxis=dict(title_text="Cantidad",  titlefont=dict(size=18))
)
#fig.show()
display(SVG(fig.to_image(format='svg')))

Vemos que se presentan 3 picos en la gráfica: A inicios, mitad y fin de año, lo cual puede estar relacionado a las temporadas de verano e invierno en Estados Unidos, en éstas épocas se concentran la mayoría de usuarios e invierten más horas de su tiempo libre en videojuegos de Steam.

In [ ]:
del users_by_month

##### -> 20 Videojuegos más antiguos

In [ ]:
older_games = data_games.drop(['url', 'categories'], axis=1).sort_values(
    by='release_date', ascending=True).head(20)

group_users = data_reviews.query('product_id in @older_games.id')\
    .groupby('product_id').product_id.count().reset_index(name='amount')

older_games = pd.merge(
    older_games, group_users, left_on='id', right_on='product_id')

older_games['year'] = older_games.release_date.dt.year

In [ ]:
fig = px.bar(data_frame=older_games,
             x='amount',
             y='year',
             #log_x=True,
             color='app_name',
             labels={'year': 'Año', 'amount': 'Cantidad de usuarios',
                     'app_name': 'Videojuego'},
             title='20 videojuegos más antiguos',
             hover_data=HOVER_DATA_COLUMNS, orientation='h')

fig.update_layout(
    width=850,
    height=550,
    paper_bgcolor='#FAFAFA',
    legend=dict(traceorder='reversed'),
    margin=dict(t=50),
)
#fig.show()
display(SVG(fig.to_image(format='svg')))

El videojuego "Dragon's Lair" a pesar de ser el más antiguo es el que posee mayor cantidad de reviews entre los videojuegos de esa época.

In [ ]:
del older_games, group_users

#### Nube de palabras Videojuego más popular

Inicializando variables:

In [ ]:
reviews = data_reviews[data_reviews.product_id ==
                       reviews_games_amount.iloc[0].id].text
stopwords = nltk.corpus.stopwords.words('english')
wnl = WordNetLemmatizer()
sentences=[]

In [ ]:
for review in reviews:
    # Normalizando
    sent = re.sub('[^a-zA-Z!¡?¿]', ' ', review).strip().lower()
    words = nltk.word_tokenize(sent)
    
    # Aplicando Lematización
    words = [wnl.lemmatize(word) for word in words]
    words = [word for word in words if (
        word not in stopwords) and len(word) > 2]    
    words = " ".join(words)
    sentences.append(words)

Formando una sola oración con todas las palabras:

In [ ]:
sentences = " ".join(sentences)

Imagen original:

In [ ]:
steam_img = Image.open('background.jpg')
steam_img

Procesando imagen para la nube de palabras:

In [ ]:
steam_img = np.array(steam_img)[::3, ::3]
steam_mask = steam_img.copy()
steam_mask[steam_mask.sum(axis=2) == 88] = 255

edges = np.mean([gaussian_gradient_magnitude(steam_img[:, :, i]/255, .5)
                 for i in range(3)], axis=0)
steam_mask[edges > .200] = 255

In [ ]:
wc = WordCloud(mask=steam_mask, max_words=2800, relative_scaling=.7,
               max_font_size=50, repeat=False, scale=3, contour_color='white')
wc.generate(sentences)

Finalmente:

In [ ]:
image_color=ImageColorGenerator(steam_img)
wc.recolor(color_func=image_color)
wc.to_image()

In [ ]:
del steam_img, steam_mask, wc

## Sistema de recomendación

### Nuevos usuarios

A los nuevos usuarios se les recomendará automáticamente los videojuegos más populares, al consumirlos serán tenidos en cuenta por el algoritmo recomendador para futuras recomendaciones:

In [ ]:
def plot_table(df):
    COLUMNS_FOR_USER = ['app_name', 'url', 'category_price']
    fig = go.Figure(go.Table(
        header=dict(values=['Videojuego', 'Sitio web', 'Tipo de precio'],
                    fill_color='#ffd166',
                    font=dict(size=13)),
        cells=dict(values=np.vstack(df[COLUMNS_FOR_USER].values).T,
                   fill_color='#264653',
                   line_color='black',
                   line_width=.4,
                   font=dict(color='#f1faee'),
                   height=30),
        columnwidth=[1.5, 4, 1]))

    fig.update_layout(width=1000, height=350, margin=dict(t=10, b=0, l=0))
    return display(SVG(fig.to_image('svg')))

In [ ]:
plot_table(reviews_games_amount.head(10))

### Benchmark

Antes de comenzar con modelos avanzados de recomendaciones, un buen inicio es crear un benchmark que sea capaz de recomendar videojuegos a usuarios basado en su popularidad, recordemos que éste no tiene en cuenta características de item, usuarios ni tampoco la interacción entre ellos:

In [ ]:
def get_popularity_recommendations(username, top=10):
    df_recs = reviews_games_amount[~reviews_games_amount.id.isin(
        data_reviews.query('username==@username').product_id)]
    plot_table(df_recs.head(top))

In [ ]:
get_popularity_recommendations('Astral')

### Filtro colaborativo

La implementación de un filtro colaboratorio conlleva muchos retos, algunos de ellos se han estado solucionando en pasos anteriores con la limpieza y procesamiento de los datos, sin embargo, para el desarrollo de éste proyecto se realizó una etapa de investigación adicional para solucionar la ausencia de calificaciones entre los pares usuario-item el cual era un campo esperado cuando se importaron los datos.  
Desde luego, en el consumo de contenidos digitales los usuarios no suelen brindar una calificación explícita que cuantifique su gusto o apreciación hacia cierto producto, un hecho que nos lleva a mirar este desafio como un problema que se volvería a repetir en un caso real. 

La metodología inicial consistió en calcular éstas calificaciones manualmente y posteriormente utilizar la libreria Surprise, pero era una práctica muy tediosa y poco recomendada segun algunos foros, no obstante recomendaban utilizar las Matrices de Factorización muy útiles para aproximar valoraciones implícitas. La búsqueda arrojó una serie de librerias, las más comunes son **Implicit** y **LightFM**, ambas con el mismo enfoque.

#### Mínimos Cuadrados

La librería Implicit (https://implicit.readthedocs.io/en/latest/) ofrece muchas funcionalidades para implementar filtros colaborativos con ratings implícitos, el método de los Mínimos Cuadrados es uno de los algoritmos de las Máquinas Factoriales y fué el primero resultado de la búsqueda:

In [ ]:
from implicit.als import AlternatingLeastSquares, train_test_split
from implicit.evaluation import ranking_metrics_at_k
from scipy import sparse

Creando merge entre dataset de reviews y videojuegos para tener la categoría del precio en cada review:

In [ ]:
reviews_games = pd.merge(data_reviews, data_games[['id', 'category_price']],
                        left_on='product_id',
                        right_on='id',
                        how='left')

In [ ]:
reviews_games.head()

Convirtiendo a variable categórica el evento (Tipo de precio de videojuego) y el nombre de usuario:

In [ ]:
reviews_games['event'] = reviews_games['category_price'].astype(
    'category').cat.codes.astype(float)
reviews_games['user_code'] = reviews_games['username'].astype(
    'category').cat.codes

In [ ]:
print(reviews_games.dtypes)

Matriz de dispersión item-usuario y usuario-item:

In [ ]:
sparse_item_user = sparse.csr_matrix((reviews_games['hours'], (reviews_games['product_id'], reviews_games['user_code'])))
sparse_user_item = sparse.csr_matrix((reviews_games['hours'], (reviews_games['user_code'], reviews_games['product_id'])))

Creando el modelo:

In [ ]:
#Building the model
model = AlternatingLeastSquares(factors=20, regularization=0.1, iterations=40)

In [ ]:
alpha_val = 40
data_conf = (sparse_item_user * alpha_val).astype('double')

Generando datos de entrenamiento y validación:|

In [ ]:
train, test = train_test_split(data_conf, .7)

In [ ]:
print('Shape:', train.getnnz(), test.getnnz())

Entrenando:

In [ ]:
model.fit(train)

Evaluando:

In [ ]:
ranking_metrics_at_k(model, train, test)

Haciendo recomendaciones:

In [ ]:
user_id =  136494
recommended = model.recommend(user_id, sparse_user_item)
print(recommended)

Vemos que ademas del id del item recomendado lo acompaña la calificacion implicita calculada por la libreria.

In [ ]:
ids = [i[0] for i in recommended]
ids

In [ ]:
plot_table(data_games.query('id in @ids'))

Items similares:

In [ ]:
item_id = 239160
ids=[i[0] for i in model.similar_items(item_id)]
print(ids)
plot_table(data_games.query('id in @ids'))

In [ ]:
del  model, reviews_games, train, test

#### Matriz Factorial

En el apartado anterior, se aplicó el método de los Mínimos Cuadrados, pero fué necesario convertir algunos tipos de datos a variable numérica discreta. La librería a continuación implementa el algoritmo de la Matriz de Factorización y evita realizar transformaciones adicionales. El framework adpota una nueva versión del algoritmo para trabajar con calificaciones implícitas:

In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

##### Sin features

Implementando el modelo sin incluir las features de los videojuegos para poder evaluar su rendimiento:

* Renombrando columnas para dividir datos en entrenamiento y validación:

In [ ]:
data_reviews.rename(columns={'username': 'user_id',
                             'product_id': 'item_id', 'hours': 'weight'}, inplace=True)

In [ ]:
data_reviews.columns

* Preparando datos:

In [ ]:
data_interactions = Dataset()
data_interactions.fit(
    users=data_reviews['user_id'], items=data_reviews['item_id'], )

In [ ]:
interactions, weights = data_interactions.build_interactions(
    data_reviews[['user_id', 'item_id', 'weight']].values)

* Creando datos de entrenamiento y validación:

In [ ]:
random_state_interactions = np.random.RandomState(seed=42)
train, test = random_train_test_split(
    interactions, test_percentage=.2, random_state=random_state_interactions)

In [ ]:
random_state_weights = np.random.RandomState(seed=42)
train_weights, test_weights = random_train_test_split(
    interactions=weights, test_percentage=.2, random_state=random_state_weights)

In [ ]:
print('Shapes:', train.getnnz(), test.getnnz())

* Creando el modelo:

In [ ]:
lightFM = LightFM(loss='logistic')

* Entrenamiento:

In [ ]:
lightFM.fit(train, sample_weight=train_weights, verbose=True, epochs=30)

6. Evaluación:

In [ ]:
precision_at_k = precision_at_k(lightFM, test_interactions=test, train_interactions=train).mean()
print('Precision at k: %.7f' % precision_at_k)

In [ ]:
auc_score = auc_score(lightFM, test_interactions=test, train_interactions=train).mean()
print('AUC Score: %.7f' % auc_score)

In [ ]:
recall_at_k = recall_at_k(lightFM, test_interactions=test, train_interactions=train).mean()
print('Recall at k: %.7f' % recall_at_k)

##### Incluyendo features de usuario e items

En ésta etapa, el modelo se alimentará de las features de los videojuegos, actualmente contamos con el tipo de precio y categorías, para ello crearemos sus variables de dummy:

* Categorías:

In [ ]:
all_categories = np.unique(list(itertools.chain(*data_games.categories)))
len(all_categories)

In [ ]:
item_features = list(itertools.chain(*[[f'{f}:0', f'{f}:1'] for f in all_categories]))
len(item_features)

In [ ]:
unique_price = np.unique(data_games.category_price)
for price in unique_price:
    item_features.append(f'category_price:{price}')
len(item_features)

In [ ]:
data_with_features = Dataset()

data_with_features.fit(users=data_reviews['user_id'],
                       items=data_reviews['item_id'],
                       item_features=item_features
                       )

* Interacciones:

In [ ]:
# Dejando solo las variables dummy y el id de cada videojuego
games_with_reviews = data_games.query('id in @data_reviews.item_id').copy()

In [ ]:
for cat in all_categories:
    games_with_reviews[cat] = games_with_reviews.categories.apply(
        lambda x: 1 if cat in x else 0)

In [ ]:
games_id = games_with_reviews.id

In [ ]:
games_with_reviews.drop(
    ['id', 'publisher', 'genres', 'app_name', 'url',
        'release_date', 'categories'],
    axis=1, inplace=True)

In [ ]:
games_with_reviews.columns

In [ ]:
def get_col_features(features):
    result = []
    for col, feature in zip(games_with_reviews.columns, features):
        result.append(f'{col}:{feature}')
    return result

In [ ]:
interactions_features = [i for i in games_with_reviews.values]
pair_interactions_features = []
for features in interactions_features:
    pair_interactions_features.append(get_col_features(features))

pair_interactions_features = list(zip(games_id, pair_interactions_features))

In [ ]:
# Creando interacciones
interactions, weights = data_with_features.build_interactions(data_reviews[['user_id', 'item_id', 'weight']].values)

# Transformando features
item_features = data_with_features.build_item_features(pair_interactions_features)

* Creando datos de entrenamiento y validación:

In [ ]:
random_state_interactions = np.random.RandomState(seed=42)

train, test = random_train_test_split(
    interactions, test_percentage=.2, random_state=random_state_interactions)

In [ ]:
random_state_weights = np.random.RandomState(seed=42)

train_weights, test_weights = random_train_test_split(
    interactions=weights, test_percentage=.2, random_state=random_state_weights)

In [ ]:
print('Shapes Interactions:', train.getnnz(), test.getnnz())
print('Shapes Weights:', train_weights.getnnz(), test_weights.getnnz())

* Creando el modelo:

In [ ]:
lightFM_with_features = LightFM(loss='logistic')

* Entrenamiento:

In [ ]:
lightFM_with_features.fit(
    train, item_features=item_features, verbose=True, epochs=2)

* Evaluación:

In [ ]:
precision_at_k_train = precision_at_k(
    lightFM_with_features, train, item_features=item_features).mean()
print('Precision at k (Train): %.7f' % precision_at_k_train)

In [ ]:
auc_score = auc_score(lightFM_with_features, test_interactions=test, train_interactions=train, 
                      item_features=test_features).mean()
print('AUC Score: %.7f' % auc_score)

In [ ]:
recall_at_k = recall_at_k(lighlightFM_with_featurestFM, test_interactions=test, train_interactions=train, 
                          item_features=test_features).mean()
print('Recall at k: %.7f' % recall_at_k)